# Business KPI Analysis Using SQL

**Objective:**  
Use SQL to compute key business metrics related to sales performance, seasonality, and risk. These queries replicate questions a business stakeholder or manager might ask when making operational decisions.


##: Questions

Q1. WHat are total and average weekly sales?

Q2. Which stores contribute the most and least to total revenue?

Q3. How do Holiday and Non-Holiday sales compare?

Q4. Which stores are the most volatile?

Q5. How does volatility differ during the holidays?

In [29]:
#import libraries
import sqlite3
import pandas as pd

In [30]:
conn = sqlite3.connect('..\db\walmart.db')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\david\AppData\Local\Temp\ipykernel_28824\3339318204.py:1: SyntaxWarning: invalid escape sequence '\d'
  conn = sqlite3.connect('..\db\walmart.db')


In [31]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type ='table';",
    conn
)

,name
0,walmart_sales


In [32]:
pd.read_sql(
    "SELECT * FROM walmart_sales LIMIT 5",
    conn
)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05 00:00:00,24924.50,0,A,151315,42.31,2.572,None,None,None,None,None,211.096358,8.106
1,1,1,2010-02-12 00:00:00,46039.49,1,A,151315,38.51,2.548,None,None,None,None,None,211.242170,8.106
2,1,1,2010-02-19 00:00:00,41595.55,0,A,151315,39.93,2.514,None,None,None,None,None,211.289143,8.106
3,1,1,2010-02-26 00:00:00,19403.54,0,A,151315,46.63,2.561,None,None,None,None,None,211.319643,8.106
4,1,1,2010-03-05 00:00:00,21827.90,0,A,151315,46.50,2.625,None,None,None,None,None,211.350143,8.106


## 1) Overall Sales Performance (Baseline KPIs)

In [33]:
pd.read_sql("""
SELECT ROUND(SUM(Weekly_Sales)/1000000000,4) as Total_Sales_Ten_Billions,
        Round(AVG(Weekly_Sales),2) as Average_Waekly_Sales
from walmart_sales
""",
conn
)

,Total_Sales_Ten_Billions,Average_Waekly_Sales
0,6.7372,15981.26


### 1.1) Top 10 Performing Stores (Total Sales)

In [48]:
pd.read_sql("""
SELECT 
    Store,
    ROUND(SUM(Weekly_Sales)/100000000,2) as Total_Sales_Hundred_Million
FROM walmart_sales
GROUP BY Store
ORDER BY Total_Sales_Hundred_Million DESC
LIMIT 10
""",
conn)

,Store,Total_Sales_Hundred_Million
0,20,3.01
1,4,3.00
2,14,2.89
3,13,2.87
4,2,2.75
5,10,2.72
6,27,2.54
7,6,2.24
8,1,2.22
9,39,2.07


**Insight**: The top-performing stores generate total sales nearly five times larger than bottom-ranked stores. This disparity appears driven by store scale rather than efficiency which will require use to examine average weekly sales and votality adjusted metrics. 

The top 10 stores generate significantly higher total sales than the rest of stores, with totals around $300k - $200k in comparison to the bottom performing stores with total sales ranging sub -$50k. This gap could be explained by other store-level factors such as size, CPI, or Fuel_Price which will be explored in future analysis.

In [52]:
pd.read_sql(
"""
Select 
Store,
AVG(Weekly_Sales) as Average_Weekly_Sales
FROM walmart_sales
GROUP BY Store
Order by Average_Weekly_Sales DESC
LIMIT 10
""",
conn
)

,Store,Average_Weekly_Sales
0,20,29508.301592
1,4,29161.210415
2,14,28784.851727
3,13,27355.136891
4,2,26898.070031
5,10,26332.303819
6,27,24826.984536
7,6,21913.243624
8,1,21710.543621
9,39,21000.763562


**Key Insight**: By using average weekly sales per store, we control for longevity bias across locations. The top 10 stores by total sales also appear among the top 10 in average weekly sales, indicating that their strong performance is driven by consistently high weekly revenue rather than store age alone.

## 2) Time Series Analysis — Sales Trends

### 2.1) How do average weekly sales change over time?

In [34]:
pd.read_sql("""
SELECT Date,
        ROUND(AVG(Weekly_Sales), 2) as Average_Weekly_Sales
From walmart_sales
GROUP BY Date
ORDER BY Date
""",
conn
)

,Date,Average_Weekly_Sales
0,2010-02-05 00:00:00,16836.12
1,2010-02-12 00:00:00,16352.06
2,2010-02-19 00:00:00,16216.66
3,2010-02-26 00:00:00,14899.55
4,2010-03-05 00:00:00,15921.02
...,...,...
138,2012-09-28 00:00:00,14765.33
139,2012-10-05 00:00:00,15983.41
140,2012-10-12 00:00:00,15427.60
141,2012-10-19 00:00:00,15295.73


### 2.2) What is the week-over-week change in average sales?

In [55]:
pd.read_sql("""
WITH weekly_avg AS (
SELECT Date,
        AVG(Weekly_Sales) as Average_Weekly_Sales,
        LAG(AVG(Weekly_Sales)) OVER (ORDER by Date) as Last_Weeks_Sales
FROM walmart_sales
GROUP BY Date
ORDER BY Date)
            
SELECT 
        DATE(Date) AS Date,
        ROUND(Average_Weekly_Sales,2) AS Average_Weekly_Sales,
        ROUND(Last_Weeks_Sales,2) AS Last_Weeks_Sales,
        Average_Weekly_Sales - Last_Weeks_Sales as WoW_Change,
        ROUND(
            (Average_Weekly_Sales - Last_Weeks_Sales) /
            NULLIF(Last_Weeks_Sales,0)*100,2) AS WoW_Percent_Change
FROM weekly_avg

""",
conn)

,Date,Average_Weekly_Sales,Last_Weeks_Sales,WoW_Change,WoW_Percent_Change
0,2010-02-05,16836.12,NaN,NaN,NaN
1,2010-02-12,16352.06,16836.12,-484.065965,-2.88
2,2010-02-19,16216.66,16352.06,-135.397053,-0.83
3,2010-02-26,14899.55,16216.66,-1317.109291,-8.12
4,2010-03-05,15921.02,14899.55,1021.466039,6.86
...,...,...,...,...,...
138,2012-09-28,14765.33,14989.71,-224.381106,-1.50
139,2012-10-05,15983.41,14765.33,1218.086468,8.25
140,2012-10-12,15427.60,15983.41,-555.817008,-3.48
141,2012-10-19,15295.73,15427.60,-131.864343,-0.85


## 3) Store-Level Sales Votality

### 3.1) Which stores exhibit the highest variability in weekly sales?

In [62]:
pd.read_sql(
    """
WITH walmart_stats as (SELECT
Store,
AVG(Weekly_Sales) as Avg_Weekly_Sales,
ROUND(SQRT(AVG(Weekly_Sales * Weekly_Sales) - AVG(Weekly_Sales) * AVG(Weekly_Sales)),2) as STD_Weekly_Sales
FROM walmart_sales
GROUP BY Store
ORDER BY Store)
SELECT
STORE,
ROUND(STD_Weekly_Sales/Avg_Weekly_Sales,2) AS CV
FROM walmart_stats
ORDER BY CV DESC
""",
conn
)

,Store,CV
0,3,2.24
1,44,1.76
2,38,1.70
3,30,1.64
4,5,1.60
5,42,1.60
6,37,1.58
7,33,1.57
8,43,1.55
9,36,1.51
